<a href="https://colab.research.google.com/github/rafiqmyura/Learn-Machine-Learning/blob/main/Disable_Sign_People_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import Callback
import keras_cv

BATCH_SIZE = 4
GLOBAL_CLIPNORM = 10.0

AUTO = tf.data.AUTOTUNE

Using TensorFlow backend


### Using Preprocessing with TensorFlow

In [9]:
def parse_txt_annot(img_path, txt_path):
    img = cv2.imread(img_path)
    w = int(img.shape[0])
    h = int(img.shape[1])

    file_label = open(txt_path, "r")
    lines = file_label.read().split('\n')

    boxes = []
    classes = []

    for i in range(0, int(len(lines))):
        objbud=lines[i].split(' ')
        class_ = int(objbud[0])

        x1 = float(objbud[1])
        y1 = float(objbud[2])
        w1 = float(objbud[3])
        h1 = float(objbud[4])

        xmin = int((x1*w) - (w1*w)/2.0)
        ymin = int((y1*h) - (h1*h)/2.0)
        xmax = int((x1*w) + (w1*w)/2.0)
        ymax = int((y1*h) + (h1*h)/2.0)

        boxes.append([xmin ,ymin ,xmax ,ymax])
        classes.append(class_)

    return img_path, classes, boxes


# a function for creating file paths list
def create_paths_list(path):
    full_path = []
    images = sorted(os.listdir(path))

    for i in images:
        full_path.append(os.path.join(path, i))

    return full_path


class_ids = ['apart_parking', 'blackbox', 'disabled_parking', 'etc', 'highpass', 'navigator']
class_mapping = dict(zip(range(len(class_ids)), class_ids))

In [10]:
def creating_files(img_files_paths, annot_files_paths):

    img_files = create_paths_list(img_files_paths)
    annot_files = create_paths_list(annot_files_paths)

    image_paths = []
    bbox = []
    classes = []

    for i in range(0,len(img_files)):
        image_path_, classes_, bbox_ = parse_txt_annot(img_files[i], annot_files[i])
        image_paths.append(image_path_)
        bbox.append(bbox_)
        classes.append(classes_)

    image_paths = tf.ragged.constant(image_paths)
    bbox = tf.ragged.constant(bbox)
    classes = tf.ragged.constant(classes)

    return image_paths, classes, bbox

In [11]:
train_img_paths, train_classes, train_bboxes = creating_files ('/content/drive/MyDrive/Data Disable Detection/train/images', '/content/drive/MyDrive/Data Disable Detection/train/labels')

valid_img_paths, valid_classes, valid_bboxes = creating_files ('/content/drive/MyDrive/Data Disable Detection/valid/images', '/content/drive/MyDrive/Data Disable Detection/valid/labels')

test_image_paths, test_classes, test_bboxes = creating_files ('/content/drive/MyDrive/Data Disable Detection/test/images', '/content/drive/MyDrive/Data Disable Detection/test/labels')

### Building Dataset